# FleetTrack Tech Residency 40

In [ ]:
import numpy as np
import pandas as pd
import time
import json
import random

### 4 hours of data for one vehicle

In [ ]:
car1 = pd.DataFrame({"vehicle_id": ['c1'] * 24,
                    "timestamp": pd.date_range(start='2023-10-01 00:09:00', periods=24, freq='10min'),
                    "speed_mph": np.round(np.random.uniform(0, 70, 24), 2),})
print(car1)

   vehicle_id           timestamp  speed_mph
0          c1 2023-10-01 00:09:00      22.00
1          c1 2023-10-01 00:19:00      57.34
2          c1 2023-10-01 00:29:00      39.57
3          c1 2023-10-01 00:39:00      30.05
4          c1 2023-10-01 00:49:00      20.27
5          c1 2023-10-01 00:59:00      66.23
6          c1 2023-10-01 01:09:00      43.90
7          c1 2023-10-01 01:19:00      61.41
8          c1 2023-10-01 01:29:00       4.34
9          c1 2023-10-01 01:39:00       6.44
10         c1 2023-10-01 01:49:00      36.70
11         c1 2023-10-01 01:59:00      23.63
12         c1 2023-10-01 02:09:00      28.23
13         c1 2023-10-01 02:19:00      64.42
14         c1 2023-10-01 02:29:00      14.72
15         c1 2023-10-01 02:39:00      21.50
16         c1 2023-10-01 02:49:00      55.83
17         c1 2023-10-01 02:59:00      67.01
18         c1 2023-10-01 03:09:00      64.57
19         c1 2023-10-01 03:19:00      65.90
20         c1 2023-10-01 03:29:00      22.75
21        

In [ ]:
# Random Walk (simple realistic variation)
np.random.seed(42)

n = 24
timestamps = pd.date_range(start='2023-10-01 00:09:00', periods=n, freq='10min')

# Start around 30 mph and add small random changes
speeds = [30]
for _ in range(1, n):
    change = np.random.uniform(-5, 5)  # small change per step
    new_speed = max(0, min(70, speeds[-1] + change))  # keep in range [0,70]
    speeds.append(new_speed)

car1 = pd.DataFrame({
    "vehicle_id": ["c1"] * n,
    "timestamp": timestamps,
    "speed_mph": np.round(speeds, 2),
    "check_engine": [0] * (n - 2) + [1, 0] #simulate a check engine event
})
print(car1)


   vehicle_id           timestamp  speed_mph  check_engine
0          c1 2023-10-01 00:09:00      30.00             0
1          c1 2023-10-01 00:19:00      28.75             0
2          c1 2023-10-01 00:29:00      33.25             0
3          c1 2023-10-01 00:39:00      35.57             0
4          c1 2023-10-01 00:49:00      36.56             0
5          c1 2023-10-01 00:59:00      33.12             0
6          c1 2023-10-01 01:09:00      29.68             0
7          c1 2023-10-01 01:19:00      25.26             0
8          c1 2023-10-01 01:29:00      28.92             0
9          c1 2023-10-01 01:39:00      29.93             0
10         c1 2023-10-01 01:49:00      32.01             0
11         c1 2023-10-01 01:59:00      27.22             0
12         c1 2023-10-01 02:09:00      31.92             0
13         c1 2023-10-01 02:19:00      35.24             0
14         c1 2023-10-01 02:29:00      32.37             0
15         c1 2023-10-01 02:39:00      29.18            

### Location Database

In [ ]:
# Base coordinates roughly around Phoenix, AZ
base_lat = 33.45
base_lon = -112.07

# Slight random offsets for nearby locations (~a few miles)
def random_nearby_coordinates(base_lat, base_lon, delta=0.05):
    return (
        base_lat + random.uniform(-delta, delta),
        base_lon + random.uniform(-delta, delta)
    )

# Example fake addresses
streets = [
    "123 Main St",
    "456 Oak Ave",
    "789 Pine Rd",
    "101 Maple Blvd",
    "202 Elm St"
]

cities = ["Phoenix", "Tempe", "Scottsdale", "Mesa", "Glendale"]

# Generate 5 unique location dicts
locations = []
for i in range(5):
    lat, lon = random_nearby_coordinates(base_lat, base_lon)
    entry = {
        "location_id": f"Location_{i+1}",
        "address": {
            "street": streets[i],
            "city": cities[i],
            "state": "AZ",
            "zip": str(85000 + i)  # simple sequential ZIPs
        },
        "latitude": lat,
        "longitude": lon
    }
    locations.append(entry)

# Save to JSON file
with open("phoenix_locations.json", "w") as f:
    json.dump(locations, f, indent=2)

# Print preview
locations
